# Import libraries

In [1]:
import pandas as pd
import sys
import numpy as np
import random
import math

# Import data

In [2]:
df1 = pd.read_excel('data/Dane_S2_50_10.xlsx')
df1 = df1.drop('Zadanie',1)
df1.name = "Dane50x10"

df2 = pd.read_excel('data/Dane_S2_100_20.xlsx')
df2 = df2.drop('Zadanie',1)
df2.name = "Dane100x20"


df3 = pd.read_excel('data/Dane_S2_200_20.xlsx')
df3 = df3.drop('Zadanie',1)
df3.name = 'Dane200x20'

# Functions

In [3]:
def CountScore(df):
    m_czasy = []
    v_czasy = []

    for i in range(len(df)):
        m_czasy.append([0] * len(df.columns))

    for i in range(len(df)):
        for j in range(len(df.columns)):
            if(i!=0 and j!=0):
                czas = max( m_czasy[i-1][j] + df.iloc[i,j],m_czasy[i][j-1] + df.iloc[i,j]) 
                m_czasy[i][j] = czas
            elif(i==0 and j==0):
                czas = df.iloc[i,j]
                m_czasy[i][j] = czas
            elif(i==0):
                czas = max(df.iloc[i,j],m_czasy[i][j-1] + df.iloc[i,j])
                m_czasy[i][j] = czas
            elif(j==0):
                czas = max(df.iloc[i,j],m_czasy[i-1][j] + df.iloc[i,j])
                m_czasy[i][j] = czas

    return m_czasy[i][j]

def Cross(assigment1,assigment2,idx=5):
    idx=random.randrange(0,len(assigment1),1)
    #print(idx)

    a=assigment1[:idx], assigment1[idx:]
    b=assigment2[:idx], assigment2[idx:]
    
    child1_part1 = a[0]
    child1_part2 = a[1]
    child2_part1 = b[0]
    child2_part2 = b[1]

    child1 = child1_part1 + [ x for x in child2_part2 if not x in child1_part1]
    child1 = child1 + [ x for x in child1_part2 if not x in child1]
    
    child2 = child2_part1 + [ x for x in child1_part2 if not x in child2_part1]
    child2 = child2 + [ x for x in child1_part1 if not x in child2]
    
    return child1, child2
  
def CrossDf(df1,df2,idx=5):
    idx=random.randrange(0,len(df1),1)
    #print(idx)
    
    a=df1[:idx], df1[idx:]
    b=df2[:idx], df2[idx:]

    child1_part1 = list(a[0].index) 
    child1_part2 = list (a[1].index)
    
    child2_part1= list(b[0].index) 
    child2_part2 = list(b[1].index) 
    
    child1 = child1_part1 + [ x for x in child2_part2 if not x in child1_part1]
    child1 = child1 + [ x for x in child1_part2 if not x in child1]
    
    child2 = child2_part1 + [ x for x in child1_part2 if not x in child2_part1]
    child2 = child2 + [ x for x in child1_part1 if not x in child2]
    
    return df1.iloc[child1], df1.iloc[child2]

def CrossAlt(lis1,lis2):
    a=math.floor(len(lis1)/3*2)
    b=math.floor(len(lis1)/3)
    
    X1 = lis1[:a]
    Y1 = lis1[a:b]
    Z1 = lis1[b:]
    X2 = lis2[:a]
    Y2 = lis2[a:b]
    Z2 = lis2[b:]

    return  Z1+X2+Y1, Y2+X1+Z2 

def Mutation(li):
    li_cp = li
    rndm1 = random.randrange(0,len(li),1)
    rndm2 = random.randrange(0,len(li),1)
    tmp = li_cp[rndm1]
    li_cp[rndm1] = li_cp[rndm2]
    li_cp[rndm2] = tmp
    return li_cp
    
def MutationAlt(li):
    li_cp = li
    rndm1 = random.randrange(0,len(li),1)
    rndm2 = random.randrange(0,len(li),1)
    
    if(rndm1>=rndm2):
        a=li_cp[rndm2:rndm1]
        a.reverse()
        li_cp[rndm2:rndm1] = a
    else:
        a=li_cp[rndm1:rndm2]
        a.reverse()
        li_cp[rndm1:rndm2] = a  
    return li_cp

def Ranking(temp,n_top=50):
    temp = temp.sort_values('score')
    temp = temp[:n_top] #bierzemy top 50 najlpszych
    temp = temp.reset_index(drop=True) 
    return temp
    
def Tournament(temp):
    winners = []
    temp_bc = temp
    while(len(temp)>0):
        knights = list(temp.sample(n=2).index)
        if(temp.loc[knights[0]].score < temp.loc[knights[1]].score):
            winners.append(knights[0])
        else:
            winners.append(knights[1])
        temp = temp.drop(knights)
        
    temp = temp_bc.loc[winners].reset_index(drop=True)
    return temp


def shuffleACopy(x):
        b = x[:] # make a copy of the keys
        random.shuffle(b) # shuffle the copy
        return b # return the copy

# Main function

In [4]:
def Main(df, method='trmt',cross='standard',iterations=1, n_pop=100, n_top=50, n_generations=5,mutation_p=.2):
    results = []
    for iteration in range(iterations):
        population = [shuffleACopy(list(df.index)) for x in range(n_pop)] 

        generation_records_mean = []

        for generation in range(n_generations):

            temp = pd.DataFrame(columns=['score','assignment'])

            for i,pop in enumerate(population):
                score = CountScore(df.iloc[pop])
                temp.loc[i] = [score, pop]

            best_score = temp.sort_values('score')[:1]    

            if(method=='trmt'):
                temp = Tournament(temp)
            elif(method=='rnk'):
                temp = Ranking(temp,n_top)
            else:
                print("error: no method selected,\nproceeding with random population")

            r1 = temp[temp.index%2 == 0] #rodzic1
            r2 = temp[temp.index%2 == 1].reset_index(drop=True) #rodzic2

            new_pop =[]

            for i,rs1 in enumerate(r1.assignment):
                if(cross=='alt'):
                    children = CrossAlt(rs1, r2.assignment[i])
                else:
                    children = Cross(rs1, r2.assignment[i])
                if(random.random()>(1-mutation_p) ):
                    if(random.random()>.5):
                        Mutation(children[0])
                    else:
                        MutationAlt(children[0])
                if(random.random()>(1-mutation_p) ):
                    if(random.random()>.5):
                        Mutation(children[1])
                    else:
                        MutationAlt(children[1])
                new_pop.append(children[0])
                new_pop.append(children[1])

            population = temp.assignment.tolist() + new_pop
            if( int(temp.sort_values('score')[:1].score) > int(best_score.score) ):
                best_score = temp.sort_values('score')[:1]
            generation_records_mean.append(temp.score.mean())

        d={'best_score':int(best_score.score),
           'iteration':iteration,
          'dataset':df.name,
          'n_pop:':n_pop,
           'method':method,
           'cross':cross,
          'generations': n_generations,
           'mutation_p':mutation_p,
          'best assignment':list(best_score.assignment),
          'mean for every gen':generation_records_mean}
        results.append(d)
    return results

def GetRecordFromResults(results):
    record = {'best_score':sys.maxsize}
    for res in results:
        if(res['best_score']<record['best_score']):
            record = res
    return record

# Executing

In [ ]:
%%time

###############>parametry<###################
iterations = 15
n_pop = 100 #liczba osobnikow w populacji
n_top = 50 #ile najlpeszych
n_generations = 30 #ile generacji
df = df3 #ktory zbior bierzemy
method = 'trmt'
# method = 'rnk'
# cross = 'standard'
cross = 'alt'
mutation_p = 0.3
#############################################

results = Main(df, method,cross,iterations, n_pop, n_top, n_generations,mutation_p)
record = GetRecordFromResults(results)
    
print("----->",record['best_score'])

Run cell below to save record to txt

In [19]:
%%capture cap --no-stderr

print(record) 
with open(str('outputs//output_'+record['dataset']+'_'+str(record['best_score'])+'.txt'),
          'w') as f:
    f.write(cap.stdout)

# Notes

In [7]:
# abc = [11,12,13]
# abc + abc
# len([5, 1, 2]+[1,2])
# [ x for x in list(Cross(df,df,5)[1].index) if not x in abc]
# Cross(df2, df2.sample(n=len(df2)), 5)[0]
# df1.sample(n=len(df1))
# Cross(df, df.sample(n=len(df)), 5)[0]

# zzz=list(df.index)
# random.shuffle(zzz)
# Cross(list(df.index),zzz)[0]